# Demo Code: Export&Import Parameter Sets

2 methods below to export and import parameter sets.

## Prerequisite
Create your own yaml file based on [configuration_template.yaml](./configuration_template.yaml), and modify "CONFIG_FILE" below

In [1]:
import sys
import os
import yaml
import subprocess

In [2]:
# point to a local file with credentials. It is not synced to git.
CONFIG_FILE = "export_import_parameter_set.yaml"

In [3]:
# Load parameters from the YAML file
with open(CONFIG_FILE, 'r') as file:
    config = yaml.safe_load(file)

# print(config)
DSJOB_URL = config['url']
DSJOB_USER = config['user']
DSJOB_PWD = config['password']
EXPORT_PRJ_NAME = config['export_prj_name']
if "export_zip_name" in config:
    EXPORT_ZIP_NAME = config['export_zip_name']
else:
    EXPORT_ZIP_NAME = EXPORT_PRJ_NAME+"_ds_wp_prj.zip"

parameter_sets = config['parameter_sets']

IMPORT_PRJ_NAME = config['import_prj_name']
print("EXPORT_PRJ_NAME=",EXPORT_PRJ_NAME)
print("IMPORT_PRJ_NAME=",IMPORT_PRJ_NAME)

EXPORT_PRJ_NAME= Multicloud Data Integration L3 Tech Lab
IMPORT_PRJ_NAME= DataStage Import


In [4]:
# Enable dsjob
%env CPDCTL_ENABLE_DSJOB=true
%env CPDCTL_ENABLE_DATASTAGE=true
%env CPDCTL_ENABLE_VOLUMES=1

env: CPDCTL_ENABLE_DSJOB=true
env: CPDCTL_ENABLE_DATASTAGE=true
env: CPDCTL_ENABLE_VOLUMES=1


In [5]:
# Configure cpdctl with the parameters
!cpdctl config user set CP4D-user --username "$DSJOB_USER" --password "$DSJOB_PWD"
!cpdctl config profile set CP4D-profile --url "$DSJOB_URL" --user CP4D-user
!cpdctl config profile use CP4D-profile

Switched to profile "CP4D-profile".


In [6]:
# list all projects
!cpdctl dsjob list-projects

...
DataStage Import
test_python
Multicloud Data Integration L3 Tech Lab
test_python_pipeline

Total: 4 Projects

Status code = 0


## Method 1: Using **"export-paramset"**  and  **"import-zip"** command
This method will export and import specified parameter sets' all values and their value sets with a ZIP file.

In [7]:
# Format the string for the export-paramset
parameter_sets_str=''
if len(parameter_sets)>0:
    parameter_sets_str = ' '.join(f'--name "{parameter_set}"' for parameter_set in parameter_sets)

cmd_str = f'cpdctl dsjob export-paramset --project "{EXPORT_PRJ_NAME}" \
{parameter_sets_str} --file-name "{EXPORT_ZIP_NAME}"'

print(cmd_str)

cpdctl dsjob export-paramset --project "Multicloud Data Integration L3 Tech Lab" --name "JB_parameter_set" --name "paraset2" --file-name "own_name.zip"


In [8]:
os.system(cmd_str)

...

Status code = 0


0

### Import to a project with **import-zip**

Connection between development and production environment are usually different. Hence we usually skip "connection" with "--skip-on-replace connection" flag.

In [9]:
!cpdctl dsjob import-zip --project "$IMPORT_PRJ_NAME" --file-name "$EXPORT_ZIP_NAME" --conflict-resolution replace --wait 200 

...
2024-06-03 21:21:20: Waiting until import finishes, import id: a9a0440f-8a10-459c-9bfc-f2dd7077b678
2024-06-03 21:21:31: Project import status: completed,  total: 2, completed: 0, failed: 0, skipped: 2.
Information:
	Parameter Set: JB_parameter_set,	  New parameters are identical to those in the existing parameter set `JB_parameter_set`, flow is updated to reference `JB_parameter_set`.

	Parameter Set: paraset2,	  New parameters are identical to those in the existing parameter set `paraset2`, flow is updated to reference `paraset2`.


Status code =  0


## Method 2: Using **"get-paramset"**, **"create-paramset"** and **"update-paramset"** command
**"get-paramset"** can ONLY ONE  specified parameter set and its all value sets to a JSON file

In [10]:
#cpdctl dsjob create-paramset {--project PROJECT | --project-id PROJID} --name NAME [--description DESCRIPTION] [--file-name FILENAME] [--param type:name:[value]...]
!cpdctl dsjob get-paramset --project "$EXPORT_PRJ_NAME" --name "JB_parameter_set" --file-name "JB_parameter_set.json"
!cat "JB_parameter_set.json"

...
ParamSet: JB_parameter_set(8cd7157f-9ffc-4e46-af64-405d8622d126) 
Name                           |Type  |Default|Prompt
----                           |----  |-------|------
$APT_AGGREGATOR_NULLABLE_OUTPUT|enum  |True   |Aggregator nullable output
JB_para1                       |string|hello  |hello

ValueSet: value_set1
Name                           |Default
----                           |-------
$APT_AGGREGATOR_NULLABLE_OUTPUT|True
JB_para1                       |hello

ValueSet: value_set2
Name                           |Default
----                           |-------
$APT_AGGREGATOR_NULLABLE_OUTPUT|False
JB_para1                       |world

Status code = 0
{
    "parameter_set": {
        "description": "",
        "name": "JB_parameter_set",
        "parameters": [
            {
                "name": "$APT_AGGREGATOR_NULLABLE_OUTPUT",
                "prompt": "Aggregator nullable output",
                "subtype": "",
                "type": "enum",
                "va

In [11]:
!cpdctl dsjob list-paramsets --project "$IMPORT_PRJ_NAME" --output json

[
  {
    "id": "8beac27c-3f92-4457-88b4-013d438b0d66",
    "name": "paraset2",
    "created_at": "2024-06-03T11:15:04.000Z",
    "updated_at": "2024-06-03T11:15:04Z"
  },
  {
    "id": "6c546320-504c-422b-831b-a8531a2e8fe2",
    "name": "JB_parameter_set",
    "created_at": "2024-06-03T11:15:04.000Z",
    "updated_at": "2024-06-03T11:20:59Z"
  }
]


In [12]:
# try  create, if failed when paramset exist, try update
!cpdctl dsjob delete-paramset --project "$IMPORT_PRJ_NAME" --name "JB_parameter_set"

...
Deleted Paramset:  JB_parameter_set

Status code =  0


In [13]:
# try  create, if failed when paramset exist, try update
!cpdctl dsjob create-paramset --project "$IMPORT_PRJ_NAME" --name "JB_parameter_set" --file-name "JB_parameter_set.json"

...
20edab71-016a-4cbd-9034-bf3dc1144bcc

Status code = 0


In [14]:
#cpdctl dsjob create-paramset {--project PROJECT | --project-id PROJID} --name NAME [--description DESCRIPTION] [--file-name FILENAME] [--param type:name:[value]...]
!cpdctl dsjob update-paramset --project "$IMPORT_PRJ_NAME" --name "JB_parameter_set" --file-name "JB_parameter_set.json"

...
ParameterSet updated for Paramset ID:  20edab71-016a-4cbd-9034-bf3dc1144bcc

Status code = 0
